In [1]:
import osmnx as ox
from IPython.display import Image
from make_fishnet import make_fishnet
import shapely.speedups
import geocoder
from glob import glob
import geopandas as gpd
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
def get_polygon(string):
    
    lis = glob('data/AOI/*/*.shp')
    
    city = []
    for i in lis:
        x = i.split('/')[-2]
        city.append(x.split('_')[0])
        
    if string in city:
        l = glob('data/AOI/{a}/*.shp'.format(a=string))
        adm = gpd.GeoDataFrame.from_file(l[0])
        boundary_poly = adm.geometry.values[0]
    else:
        boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=1)
        boundary_poly = boundary_GDF.loc[0,'geometry']
        if boundary_poly.geom_type == 'Polygon':
            boundary_poly = boundary_poly
        else:
            try:
                boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=2)
                boundary_poly = boundary_GDF.loc[0,'geometry']
            except:
                return -1
    
    return boundary_poly

In [3]:
def get_graph(place):
    string = place.split(',')[0]
    
    print('Fetching graph data for {}'.format(place))
    
    poly = get_polygon(string)
    
    if poly == -1:
        gdf = ox.gdf_from_place('{}'.format(string))
        G = ox.graph_from_bbox(gdf.bbox_north, gdf.bbox_south, gdf.bbox_east, gdf.bbox_west)
    else:
        G = ox.graph_from_polygon(poly, network_type='drive')

    G = ox.project_graph(G)
    
    return G

In [10]:
def get_node_count(place_name, pixel_size=None):
    
    ## Setting initial crs
    crs = {'init':'epsg:4326'}
    
    string = place_name.split(',')[0]
    
    print('Fetching network data from OSM for {}'.format(place_name))
    
    ## Grabbing data from OSM
    G = get_graph(place_name)
    
    gdf_proj = ox.graph_to_gdfs(G, nodes=True, edges=False)
    
    gdf = ox.gdf_from_place('{}'.format(string))
    
    if pixel_size is not None:
        size = pixel_size
    else:
        size = 500
    
    print('Creating fishnet')
    
    ## Creating fishnet and exporting the file
    try:
        os.mkdir('data/grid_data')
    except FileExistsError:
        pass
    
    make_fishnet('data/grid_data/Road_grid_{}.shp'.format(string), gdf.bbox_west,  gdf.bbox_east, gdf.bbox_south,
                 gdf.bbox_north, size, size)
    grid = gpd.GeoDataFrame.from_file('data/grid_data/Road_grid_{}.shp'.format(string))
    grid.crs = {'init':'epsg:4326'}
    grid = grid.to_crs(gdf_proj.crs)
    
    
    gdf_proj = gdf_proj.reset_index()
    
    ## Cleaning up the data by rmeoving invalid geometries
    
    gdf_proj = gdf_proj.rename(columns ={'index':'id'})
    gdf_proj['geomType'] = gdf_proj.geom_type
    gdf_proj = gdf_proj[gdf_proj['geomType'] != 'GeometryCollection']
    
    print('Merging datasets and calculating the count of nodes in each pixel')
    
    merged = gpd.sjoin( grid, gdf_proj, how='left', op='intersects')
    grp = merged.groupby('FID').count()
    
    grid['node_count'] = grp.id
    
    try:
        os.mkdir('data/{}'.format(string))
    except FileExistsError:
        pass

    grid.to_file('data/{a}/{a}_roads_fishnet.shp'.format(a=string)) 

In [11]:
%%time

get_node_count('Piedmont, California')

Fetching network data from OSM for Piedmont, California
Fetching graph data for Piedmont, California
Creating fishnet
Merging datasets and calculating the count of nodes in each pixel
CPU times: user 5.83 s, sys: 94.2 ms, total: 5.93 s
Wall time: 12.5 s


In [14]:
%%time

get_node_count('Addis Ababa, Ethiopia')

Fetching network data from OSM for Addis Ababa, Ethiopia
Fetching graph data for Addis Ababa, Ethiopia
Creating fishnet
Merging datasets and calculating the count of nodes in each pixel
CPU times: user 14min 11s, sys: 6.58 s, total: 14min 18s
Wall time: 14min 24s
